In [1]:
%pip install openpyxl

%pip install xlrd

import pandas as pd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

file_name = "courses_master.xlsx"
try:
    df = pd.read_excel(file_name, sheet_name="Exam Preferences", skiprows=[0])
    print("Data successfully loaded into DataFrame.")

    new_columns = [
        "Code", "Title", "Faculty1", "Faculty2", "Exam Type", "Duration (Hrs)",
        "2025-03-03 00:00:00", "2025-03-04 00:00:00", "2025-03-05 00:00:00",
        "2025-03-06 00:00:00", "2025-03-07 00:00:00", "2025-03-08 00:00:00",
        "Blank", 
        "09:30:00", "10:30:00", "11:30:00", "14:00:00", "15:00:00", "16:00:00", "17:00:00"
    ]

    num_new_cols = len(new_columns)
    if df.shape[1] < num_new_cols:
        raise Exception("The loaded DataFrame has fewer columns than expected.")
    df = df.iloc[:, :num_new_cols]

    df.columns = new_columns

    df.to_csv("exam_master.csv", index=False)
    print("Filtered data successfully saved to 'exam_master.csv'.")
    
except Exception as e:
    print(f"An error occurred: {e}")


Data successfully loaded into DataFrame.
Filtered data successfully saved to 'exam_master.csv'.


/home/sarvesh/miniconda3/envs/timetabling/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
import pandas as pd

hardcode_dates = [
    "2025-03-03 00:00:00", "2025-03-04 00:00:00", "2025-03-05 00:00:00",
    "2025-03-06 00:00:00", "2025-03-07 00:00:00", "2025-03-08 00:00:00"
]
hardcode_days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
hardcode_times = ["09:30:00", "10:30:00", "11:30:00", "14:00:00", "15:00:00", "16:00:00", "17:00:00"]
file_path = "exam_master.csv"
df = pd.read_csv(file_path)

course_code_timings = {}
course_code_preftime = {}
course_code_prefdays = {}
course_code_type = {}

for _, row in df.iterrows():
    code = row["Code"]
    duration = float(row["Duration (Hrs)"]) * 60
    exam_type = str(row["Exam Type"])
    
    # Process preferred times and days for this row
    times_list = []
    days_list = []
    
    for t in hardcode_times:
        if row[t] == "Yes":
            times_list.append(t)
            
    for i in range(len(hardcode_dates)):
        if row[hardcode_dates[i]] == "Yes":
            days_list.append(hardcode_days[i])
    
    # Append duration
    if code in course_code_timings:
        course_code_timings[code].append(duration)
    else:
        course_code_timings[code] = [duration]
    
    # Append exam type
    if code in course_code_type:
        course_code_type[code].append(exam_type)
    else:
        course_code_type[code] = [exam_type]
    
    # Append preferred times
    if code in course_code_preftime:
        course_code_preftime[code].append(times_list)
    else:
        course_code_preftime[code] = [times_list]
    
    # Append preferred days
    if code in course_code_prefdays:
        course_code_prefdays[code].append(days_list)
    else:
        course_code_prefdays[code] = [days_list]

print(course_code_timings)
print(course_code_prefdays)
print(course_code_preftime)
print(course_code_type)

{'AIM 101': [120.0], 'AIM 102': [180.0], 'AIM 704': [180.0], 'AIM 821': [180.0], 'AIM 825': [180.0, 180.0], 'AIM 829': [180.0], 'AIM 831': [180.0], 'AIM 832': [180.0], 'AIM 843': [60.0], 'AIM 845': [180.0], 'AIM 846': [180.0], 'AMS 401': [180.0], 'AMS 402': [180.0], 'AMS 403': [150.0], 'AMS 404': [120.0], 'COM 605': [120.0], 'COM 837': [180.0], 'COM 863': [180.0], 'CSE 102': [180.0, 120.0], 'CSE 102 P': [150.0, 150.0], 'CSE 104': [120.0, 180.0], 'CSE 212': [120.0], 'CSE 606': [90.0], 'CSE 704': [150.0], 'CSE 718': [180.0], 'CSE 754': [180.0], 'CSE 816': [120.0], 'CSE 819': [180.0], 'CSE 824': [180.0], 'CSE 836': [180.0], 'CSE 852': [180.0], 'DAS 101': [180.0, 180.0], 'DAS 101P': [150.0, 150.0], 'DAS 605': [120.0], 'DAS 839': [120.0], 'DHS 101B': [180.0], 'DHS 108': [180.0], 'DHS 110': [120.0], 'DHS 201': [180.0, 180.0], 'DHS 305': [120.0], 'DHS 306': [120.0], 'DHS 308': [180.0], 'DHS 314': [120.0], 'DHS 315': [120.0], 'DHS 611': [180.0], 'ECE 212': [180.0], 'ECE 212P': [180.0], 'ECE 30

In [4]:
import pandas as pd

# Build exam_choice dictionary using course_code_type
exam_choice = {}

# course_code_type is assumed to be already populated from previous processing.
# For example:
# course_code_type = {
#    "AIM 101": ["Paper-Pen"],
#    "AIM 102": ["** No Exam **"],
#    "AIM 103": ["Paper-Pen", "Paper-Pen"],
#    "AIM 104": ["Paper-Pen", "** No Exam **"]
# }

for code, types in course_code_type.items():
    # Default exam choice is "Same"
    choice = "Same"
    
    # If there's only one exam type for the course
    if len(types) == 1:
        if types[0] == "** No Exam **":
            choice = "NoExam"
    else:
        # For multiple entries: if they are all the same...
        if all(t == types[0] for t in types):
            # If the same type is "** No Exam **", then mark as "NoExam"
            if types[0] == "** No Exam **":
                choice = "NoExam"
            else:
                choice = "Same"
        else:
            # If values differ, mark exam choice as "Different"
            choice = "Different"
    
    exam_choice[code] = choice

# Convert the exam_choice dictionary to a DataFrame with columns "code" and "exm choice"
df_choice = pd.DataFrame(list(exam_choice.items()), columns=["code", "exm choice"])

# Write the DataFrame to exam_choice.csv
df_choice.to_csv("exam_choice.csv", index=False)
print("exam_choice.csv written successfully.")

exam_choice.csv written successfully.


In [5]:
import pandas as pd

# Assume course_code_timings and exam_choice are already populated from previous processing.
# For demonstration, here's a sample exam_choice dictionary.
# exam_choice = {
#     "AIM 101": "Same",
#     "AIM 102": "NoExam",
#     "AIM 103": "Different",
#     "AIM 104": "Same"
# }

# Compute timing_choice from course_code_timings.
timing_choice = {}

for code, timings in course_code_timings.items():
    if len(timings) == 1:
        choice = "Same"
    else:
        # Check if all timing values are identical.
        # if all(t == timings[0] for t in timings):
        #     choice = "Same"
        # else:
        #     choice = "Different"
        choice = "Different"
    timing_choice[code] = choice

# Update exam_choice: if the current value is not "Different" or "NoExam",
# then update it with the timing_choice computed value.
for code, t_choice in timing_choice.items():
    if code in exam_choice:
        if exam_choice[code] not in ["Different", "NoExam"]:
            exam_choice[code] = t_choice
    else:
        exam_choice[code] = t_choice

# Convert the final exam_choice dictionary to a DataFrame
df_choice = pd.DataFrame(list(exam_choice.items()), columns=["code", "exam choice"])

# Write the DataFrame to exam_choice.csv
df_choice.to_csv("exam_choice.csv", index=False)
print("exam_choice.csv written successfully.")

exam_choice.csv written successfully.


In [6]:
file_name = "T2-24-25-Enrollments-after-Add-Drop-Process-v3.xls"
try:
    df = pd.read_excel(file_name, sheet_name="Course Roster Report", usecols=["Course","Section","OTHER/REGULAR-ELECTIVE/REGULAR-CORE","Course Variant","Student ID","Email ID","Batch","Primary Faculty"])
    print("Data successfully loaded into DataFrame.")
    df.to_csv("enrollment_master.csv", index=False)
    print("Filtered data successfully saved to 'enrollment_master.csv'.")
    
except Exception as e:
    print(f"An error occurred: {e}")


Data successfully loaded into DataFrame.
Filtered data successfully saved to 'enrollment_master.csv'.


In [7]:
file_name = "enrollment_master.csv"
df = pd.read_csv(file_name)
# df2 = pd.read_csv(file_name)
df1 =  pd.DataFrame()

df = df[df['OTHER/REGULAR-ELECTIVE/REGULAR-CORE'] != 'OTHER']
df1['Code'] = df['Course'].str.split('/').str[0]
df1['Title'] = df['Course'].str.split('/').str[1]
df1['Faculty'] = df['Primary Faculty']
df['CourseCode'] = df['Course'].str.split('/').str[0]
df['CourseTitle'] = df['Course'].str.split('/').str[1]

df1 = df1.drop_duplicates()
df = df.drop_duplicates()

df.drop('Course', axis=1, inplace=True)

df1.to_csv("course_master.csv", index=False)
df.to_csv("enrollment_master.csv", index=False)


In [8]:
df = pd.read_csv("enrollment_master.csv")
df['CourseCode'] = df['CourseCode'].str.strip()
df.to_csv("enrollment_master.csv", index=False)

In [9]:
file_name = "enrollment_master.csv"
df = pd.read_csv(file_name)
new_order = [
    'CourseCode',
    'CourseTitle',
    'OTHER/REGULAR-ELECTIVE/REGULAR-CORE',
    'Course Variant',
    'Student ID',
    'Batch',
    'Section',
    'Email ID',
    'Primary Faculty'
]

df = df[new_order]

# rename column Email ID to EmailID

df.rename(columns={'Email ID': 'EmailID'}, inplace=True)

df.to_csv(file_name, index=False)

In [10]:
# Add a new column called Lab in classroom_master.csv if 'LAB' string is found in the RoomNumber column

file_name = "classroom_master.csv"
df = pd.read_csv(file_name)

df['Lab'] = df['RoomNumber'].str.contains('LAB', case=False)
df.to_csv(file_name, index=False)



In [11]:
# Add a new column called DeptLab in classroom_master.csv if Lab column is True and 'R' string is found in the RoomNumber column

file_name = "classroom_master.csv"
df = pd.read_csv(file_name)

df['DeptLab'] = df['Lab'] & df['RoomNumber'].str.contains('R', case=False)
df.to_csv(file_name, index=False)

In [12]:
file_name = "enrollment_master.csv"
df = pd.read_csv(file_name)

for index,row in df.iterrows():
    if 'Sec' in row['Section']:
        df.at[index, 'Section'] = row['Section'].split('-')[-1]
    else:
        df.at[index, 'Section'] = 'A'

df.to_csv(file_name, index=False)

In [13]:
# Delete entries in 'enrollment_master.csv' file where the coursecode matches with code column in exam_choice.csv but the exam choice column is 'NoExam'ArithmeticError

df = pd.read_csv("enrollment_master.csv")

df1 = pd.read_csv("exam_choice.csv")

for index1, row1 in df1.iterrows():
    if df1.at[index1, 'exam choice'] == 'NoExam':
        print("Bye")
        # loop through all the entries of df and check if the course code matches with the code in df1 and the section is 'A' then update the CourseCode to 'CSE 102 P-A'
        for index, row in df.iterrows():
            if row['CourseCode'].strip() == row1['code'].strip():
                df.drop(index, inplace=True)

df.to_csv("enrollment_master.csv", index=False)




Bye
Bye
Bye
Bye
Bye
Bye
Bye
Bye


In [14]:
# Delete entries in 'enrollment_master.csv' file where the coursecode matches with code column in exam_choice.csv but the exam choice column is 'NoExam'ArithmeticError

df = pd.read_csv("exam_master.csv")

df1 = pd.read_csv("exam_choice.csv")

for index1, row1 in df1.iterrows():
    if df1.at[index1, 'exam choice'] == 'NoExam':
        # loop through all the entries of df and check if the course code matches with the code in df1 and the section is 'A' then update the CourseCode to 'CSE 102 P-A'
        for index, row in df.iterrows():
            if row['Code'].strip() == row1['code'].strip():
                df.drop(index, inplace=True)

df.to_csv("exam_master.csv", index=False)




In [15]:
import pandas as pd

# Read CSV files.
df = pd.read_csv("enrollment_master.csv")
df2 = pd.read_csv("exam_master.csv")
exam_choice_df = pd.read_csv("exam_choice.csv")

# Create a lookup dictionary from exam_choice.csv (strip spaces from codes).
choice_dict = {row['code'].strip(): row['exam choice'] for _, row in exam_choice_df.iterrows()}

# Loop through enrollment_master.csv rows once.
for index, row in df.iterrows():
    code = row['CourseCode'].strip()
    section = row['Section'].strip()
    # If this course code is in exam_choice and its exam choice is not "Same"
    if code in choice_dict and choice_dict[code] != "Same":
        # Avoid appending suffix if it is already updated
        if '-' not in code:
            last=""
            new_code = f"{code}-{section}"
            df.at[index, 'CourseCode'] = new_code
            # Update the corresponding code in exam_master.csv.
            for index2, row2 in df2.iterrows():
                if row2["Code"].strip() == code:
                    if last=="":
                        df2.at[index2, 'Code'] = new_code
                        last=f"{section}"
                    elif last=="A":
                        df2.at[index2, 'Code'] =f"{code}-B"
                        last=f"B"
                    else:
                        df2.at[index2, 'Code'] =f"{code}-A"
                        last=f"A"

# Write the updated data back.
df.to_csv("enrollment_master.csv", index=False)
df2.to_csv("exam_master.csv", index=False)
print("Files updated successfully.")

Files updated successfully.


In [16]:
import pandas as pd

hardcode_dates = [
    "2025-03-03 00:00:00", "2025-03-04 00:00:00", "2025-03-05 00:00:00",
    "2025-03-06 00:00:00", "2025-03-07 00:00:00", "2025-03-08 00:00:00"
]
hardcode_days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
hardcode_times = ["09:30:00", "10:30:00", "11:30:00", "14:00:00", "15:00:00", "16:00:00", "17:00:00"]
file_path = "exam_master.csv"
df = pd.read_csv(file_path)

course_code_timings = {}
course_code_preftime = {}
course_code_prefdays = {}
course_code_type = {}

for _, row in df.iterrows():
    code = row["Code"]
    duration = float(row["Duration (Hrs)"]) * 60
    exam_type = str(row["Exam Type"])
    
    # Process preferred times and days for this row
    times_list = []
    days_list = []
    
    for t in hardcode_times:
        if row[t] == "Yes":
            times_list.append(t)
            
    for i in range(len(hardcode_dates)):
        if row[hardcode_dates[i]] == "Yes":
            days_list.append(hardcode_days[i])
    
    # Append duration
    course_code_timings[code] = duration
    
    # Append exam type
    course_code_type[code] = exam_type
    
    # Append preferred times
    course_code_preftime[code] = times_list
    

    course_code_prefdays[code] = days_list

print(course_code_timings)
print(course_code_prefdays)
print(course_code_preftime)
print(course_code_type)

{'AIM 101': 120.0, 'AIM 102': 180.0, 'AIM 704': 180.0, 'AIM 821': 180.0, 'AIM 831': 180.0, 'AIM 832': 180.0, 'AIM 843': 60.0, 'AIM 845': 180.0, 'AIM 846': 180.0, 'AMS 401': 180.0, 'AMS 402': 180.0, 'AMS 403': 150.0, 'AMS 404': 120.0, 'COM 605': 120.0, 'CSE 102-A': 180.0, 'CSE 102-B': 120.0, 'CSE 102 P-A': 150.0, 'CSE 102 P-B': 150.0, 'CSE 104-A': 120.0, 'CSE 104-B': 180.0, 'CSE 212': 120.0, 'CSE 606': 90.0, 'CSE 704': 150.0, 'CSE 718': 180.0, 'CSE 754': 180.0, 'CSE 816': 120.0, 'CSE 824': 180.0, 'CSE 836': 180.0, 'DAS 101-B': 180.0, 'DAS 101-A': 180.0, 'DAS 101P-B': 150.0, 'DAS 101P-A': 150.0, 'DAS 605': 120.0, 'DAS 839': 120.0, 'DHS 101B': 180.0, 'DHS 108': 180.0, 'DHS 110': 120.0, 'DHS 201-A': 180.0, 'DHS 201-B': 180.0, 'DHS 305': 120.0, 'DHS 306': 120.0, 'DHS 308': 180.0, 'DHS 314': 120.0, 'DHS 315': 120.0, 'DHS 611': 180.0, 'ECE 212': 180.0, 'ECE 212P': 180.0, 'ECE 303': 180.0, 'ECE 303P': 180.0, 'ECE 304': 90.0, 'EGC 121-A': 180.0, 'EGC 121-B': 180.0, 'EGC 123-A': 120.0, 'EGC 123-

In [17]:
def get_duration(course_code):
    if course_code in course_code_timings:
        return int(course_code_timings[course_code])
    return None

In [18]:
import pandas as pd
import numpy as np
# import data_utils as data
# Read the course_master.csv file
df = pd.read_csv("exam_master.csv")
n = len(df)

# Keep exam duration assignments as before. (They can be random.)
# indices = df.index.to_list()
# np.random.shuffle(indices)
# count_180 = int(0.7 * n)
# count_120 = int(0.2 * n)
# count_90  = n - count_180 - count_120
# df.loc[indices[:count_180], "Exam Duration"] = 180
# df.loc[indices[count_180:count_180+count_120], "Exam Duration"] = 120
# df.loc[indices[count_180+count_120:], "Exam Duration"] = 90

# Define the days and times lists
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri','Sat']
times = ['08:00', '08:15', '08:30', '08:45',
         '09:00', '09:15', '09:30', '09:45',
         '10:00', '10:15', '10:30', '10:45',
         '11:00', '11:15', '11:30', '11:45',
         '12:00', '12:15', '12:30', '12:45', '13:00',
         '14:00', '14:15', '14:30', '14:45',
         '15:00', '15:15', '15:30', '15:45',
         '16:00', '16:15', '16:30', '16:45',
         '17:00', '17:15', '17:30', '17:45', '18:00']

lunch_time_index = times.index('13:00')
pre_lunch_times = list(range(lunch_time_index + 1))
post_lunch_times = list(range((lunch_time_index + 1), len(times)))

def begin_end_pairs(session_len: int) -> list:
    pairs = []
    for s in range(0, len(times) - session_len):
        e = s + session_len
        if (((s in pre_lunch_times) and (e in pre_lunch_times)) or
            ((s in post_lunch_times) and (e in post_lunch_times))):
            pairs.append((s, e))
    return pairs

def time_to_minutes(time_str):
    hh, mm = map(int, time_str.split(":"))
    return hh * 60 + mm

# For each course, compute and print all candidate triplets.
for idx, row in df.iterrows():
    duration = get_duration(row["Code"])
    session_len = duration // 15              
    pairs = begin_end_pairs(session_len)
    
    candidate_triplets = []
    for s, e in pairs:
        begin_time = times[s]
        end_time = times[e]
        if time_to_minutes(end_time) - time_to_minutes(begin_time) == duration:
            for day in days:
                candidate_triplets.append((begin_time, end_time, day))
    
    print(f"Course {idx} with exam duration {duration} mins:")
    print(candidate_triplets)
    print("=" * 60)

Course 0 with exam duration 120 mins:
[('08:00', '10:00', 'Mon'), ('08:00', '10:00', 'Tue'), ('08:00', '10:00', 'Wed'), ('08:00', '10:00', 'Thu'), ('08:00', '10:00', 'Fri'), ('08:00', '10:00', 'Sat'), ('08:15', '10:15', 'Mon'), ('08:15', '10:15', 'Tue'), ('08:15', '10:15', 'Wed'), ('08:15', '10:15', 'Thu'), ('08:15', '10:15', 'Fri'), ('08:15', '10:15', 'Sat'), ('08:30', '10:30', 'Mon'), ('08:30', '10:30', 'Tue'), ('08:30', '10:30', 'Wed'), ('08:30', '10:30', 'Thu'), ('08:30', '10:30', 'Fri'), ('08:30', '10:30', 'Sat'), ('08:45', '10:45', 'Mon'), ('08:45', '10:45', 'Tue'), ('08:45', '10:45', 'Wed'), ('08:45', '10:45', 'Thu'), ('08:45', '10:45', 'Fri'), ('08:45', '10:45', 'Sat'), ('09:00', '11:00', 'Mon'), ('09:00', '11:00', 'Tue'), ('09:00', '11:00', 'Wed'), ('09:00', '11:00', 'Thu'), ('09:00', '11:00', 'Fri'), ('09:00', '11:00', 'Sat'), ('09:15', '11:15', 'Mon'), ('09:15', '11:15', 'Tue'), ('09:15', '11:15', 'Wed'), ('09:15', '11:15', 'Thu'), ('09:15', '11:15', 'Fri'), ('09:15', '11:15

In [19]:
# pref_time = ['09:30','10:30','11:30','14:00','15:00','16:00','17:00']
# pref_days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']

def func(begin_time,pref_time):
    for x in pref_time:
        # print("x",x)
        # strip the last 2 zeroes from x
        x = x[:-3]
        b1=time_to_minutes(begin_time)
        p1=time_to_minutes(x)
        if abs(b1-p1)<=15:
            return True
        
    return False

preferred_timings = []
for idx, row in df.iterrows():
    # duration = int(row["Exam Duration"])  
    duration = get_duration(row["Code"]) 
    session_len = duration // 15                
    pairs = begin_end_pairs(session_len)
    pref_time = course_code_preftime[row["Code"]]
    pref_days = course_code_prefdays[row["Code"]]
    # print(pref_time)
    # print(pref_days)
    # print(row["Code"])
    candidate_triplets = []
    for s, e in pairs:
        begin_time = times[s]
        end_time = times[e]
        if time_to_minutes(end_time) - time_to_minutes(begin_time) == duration:
            for day in days:
                if func(begin_time,pref_time) and day in pref_days:
                    candidate_triplets.append((begin_time, end_time, day))
    
    preferred_timings.append(candidate_triplets)

df["Preferred Timings"] = preferred_timings

df.to_csv("exam_master.csv", index=False)

In [20]:
# Write the columns [Code,Title,Faculty1,Faculty2,Duration (Hrs),Preferred Timings] in 'exam_master.csv' to 'course_master.csv'

df=pd.read_csv("exam_master.csv")
df=df[["Code","Title","Faculty1","Faculty2","Duration (Hrs)","Preferred Timings","Exam Type"]]

#Rename column Duration (Hrs) to Exam Duration and multiply the values by 60
df.rename(columns={"Duration (Hrs)":"Exam Duration"},inplace=True)
df["Exam Duration"]=df["Exam Duration"]*60

#I want to create 2 new columns 
#1. Lab - False when Exam Type is Paper-Pen else True
#2. Dept - CSE when course code has AIM , CSE, DAS else ECE

df["Lab"]=df["Exam Type"].apply(lambda x: False if "Paper-Pen" in x or "On System" in x else True)

df["Dept"]=df["Code"].apply(lambda x: "ECE" if "ECE" in x or "VLS" in x or "COM" in x else "CSE")

# Reorder the Columns 
# Preferred ordering of columns [Code,Title,Faculty1,Faculty2,Exam Duration,Lab,Dept,Exam Type,Preferred Timings]

df=df[["Code","Title","Faculty1","Faculty2","Exam Duration","Lab","Dept","Exam Type","Preferred Timings"]]

df.to_csv("course_master.csv",index=False)

In [21]:
courses = pd.read_csv('course_master.csv')
n_courses = courses.shape[0]
enrollment = pd.read_csv('enrollment_master.csv')
n_enrollments = enrollment.shape[0]
# number of unique emails in enrollment_master_clean.csv
n_students = enrollment['EmailID'].nunique()
students = enrollment['EmailID'].unique()
print(n_students)

# Write students to a csv file 
students_df = pd.DataFrame(students, columns=['EmailID'])
students_df.to_csv('students.csv', index=False)

973


In [22]:
# Read freom enrollment_master.csv and remove the blank space from the column 'CourseCode'

df = pd.read_csv("enrollment_master.csv")
df['CourseCode'] = df['CourseCode'].str.strip()
df.to_csv("enrollment_master.csv", index=False)